# Download Images

In [1]:
import sys

sys.path.append('..')

In [2]:
import multiprocessing
import shutil
import sqlite3
from pathlib import Path
from datetime import date

import pandas as pd

from herbarium.pylib import image_util as iu

In [3]:
DATA_DIR = Path('..') / 'data'

URI_DIR = DATA_DIR / 'temp'
IMAGE_DIR = DATA_DIR / 'images' / 'all'

today = date.today().isoformat()

ERROR1 = DATA_DIR / 'temp' / f'download_errors_{today}.txt'
ERROR2 = DATA_DIR / 'temp' / f'validate_errors_{today}.txt'

DB = DATA_DIR / 'angiosperms.sqlite'

## Sample records from each image class

In [4]:
iu.sample_records(DB, URI_DIR)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [04:10<00:00,  2.56s/it]


## Download images

In [5]:
with multiprocessing.Pool(processes=6) as pool:
    results = []
    for csv_file in list(URI_DIR.glob('uris_*.csv')):
        results.append(pool.apply_async(
            iu.download_images, (csv_file, IMAGE_DIR, ERROR1)))
    all_results = [result.get() for result in results]

## Validate images

In [4]:
iu.validate_images(IMAGE_DIR, DB, error=ERROR2)

15127it [1:08:02,  3.71it/s]


## Get image means and standard deviations

In [5]:
# mean, std = iu.get_image_norm(IMAGE_DIR)

100%|██████████████████████████████████████████████████████| 946/946 [1:08:58<00:00,  4.37s/it]


In [6]:
# print(f'{mean=} {std=}')

mean=tensor([0.7743, 0.7529, 0.7100]) std=tensor([0.2286, 0.2365, 0.2492])
